# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn import preprocessing

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv') 
# If you want to read a zipped or a tar.gz file into pandas dataframe, 
# the read_csv methods includes this particular implementation.
df = pd.read_csv(file_path, skiprows=1)[:-2] # [:-2] is there to omit last 2 rows that contain some supplemental information.
# loc method is used to retrieve all rows and columns listed in the last cell. Make a copy of the first dataframe.
df = df.loc[:, columns].copy() 

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all') 

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x) # replace method can accept dictionaries where the key is replaced by the value

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
# x is a dictionary that will be applied to the replace method. 
# The fromkeys method creates a dictionary using the keys as the left argument and the values as the right argument.
df = df.replace(x) 

# drop original index since columns were removed
df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
#create copy of original dataframe
df2 = df.copy()

In [6]:
# use LabelEncoder from sklearn.preprocessing to convert columns with string datatypes to numeric values and replace same column
# note: pandas get_dummies does the same but is binary and creates new columns

le = preprocessing.LabelEncoder()

for column_name, item in df2.iteritems():
    
    if type(item) != int or float:
        
        df2[column_name] = le.fit_transform(df2[column_name])
        
    else:
        
        continue
        
df2.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,348,24,3500,3,2978,1,2,1,0,2590,...,285,155,0,0,11675,19018,19,27490,0,0
1,886,28,7846,1,4688,2,2,1,0,1889,...,331,101,1,0,39916,30368,412,21865,0,0
2,703,28,4973,1,2339,2,2,1,0,2292,...,142,101,0,0,10267,23239,75,18964,0,0
3,329,22,3251,3,4252,2,2,1,0,3010,...,409,101,1,0,19544,33595,197,32527,0,0
4,775,18,4894,1,2073,0,2,1,0,1742,...,409,0,0,0,34824,11407,276,6250,0,0


In [7]:
# check that data types are all numeric for all columns
df2_types_list = list(df2.dtypes)
# df2_types_list

# Split the Data into Training and Testing

In [8]:
# Create our features
X = df2.drop(columns = target)


# Create our target
y = df2["loan_status"]
y[:5]

0    1
1    1
2    1
3    1
4    1
Name: loan_status, dtype: int32

In [9]:
X.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,348,24,3500,3,2978,1,2,0,2590,0,...,285,155,0,0,11675,19018,19,27490,0,0
1,886,28,7846,1,4688,2,2,0,1889,0,...,331,101,1,0,39916,30368,412,21865,0,0
2,703,28,4973,1,2339,2,2,0,2292,0,...,142,101,0,0,10267,23239,75,18964,0,0
3,329,22,3251,3,4252,2,2,0,3010,0,...,409,101,1,0,19544,33595,197,32527,0,0
4,775,18,4894,1,2073,0,2,0,1742,0,...,409,0,0,0,34824,11407,276,6250,0,0


In [10]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,572.296903,12.982577,4126.615168,1.812779,3294.087856,0.669994,0.805542,0.0,1962.376855,0.217722,...,367.820219,57.069314,0.125972,0.0,27710.412107,24184.046704,295.617696,19966.952715,0.0,0.0
std,375.779218,9.508701,2367.621789,0.941313,1574.576387,0.719105,0.714932,0.0,971.922012,0.717338,...,67.053040,55.878931,0.336732,0.0,17546.728857,14497.653994,257.292302,13661.709989,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,289.000000,5.000000,2153.000000,1.000000,1930.000000,0.000000,0.000000,0.0,1255.000000,0.000000,...,347.000000,0.000000,0.000000,0.0,12014.000000,11764.000000,116.000000,7895.000000,0.0,0.0
50%,516.000000,11.000000,3828.000000,1.000000,3365.000000,1.000000,1.000000,0.0,1842.000000,0.000000,...,409.000000,48.000000,0.000000,0.0,26735.000000,23056.000000,221.000000,18429.000000,0.0,0.0
75%,848.000000,20.000000,5967.000000,3.000000,4665.000000,1.000000,1.000000,0.0,2532.000000,0.000000,...,409.000000,101.000000,0.000000,0.0,42861.000000,36149.000000,393.000000,31243.000000,0.0,0.0
max,1397.000000,51.000000,8809.000000,3.000000,6089.000000,2.000000,2.000000,0.0,5695.000000,17.000000,...,409.000000,155.000000,4.000000,0.0,59628.000000,51997.000000,1715.000000,46898.000000,0.0,0.0


In [11]:
# Check the balance of our target values
Counter(y)

Counter({1: 68470, 0: 347})

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [13]:
# check shapes of each training and testing set:
print(X_train.shape, X_test.shape,  y_train.shape, y_test.shape)

(51612, 85) (17205, 85) (51612,) (17205,)


# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [14]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
# fit the data to the ros model
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print(y_resampled.shape)
print(y_train.shape)

(102732,)
(51612,)


In [15]:
print(sorted(Counter(y_resampled).items()))

[(0, 51366), (1, 51366)]


In [16]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
# fit the logreg model to the resampled training data
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [17]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
# make prediction using y_test data
y_pred = model.predict(X_test)
accuracy = balanced_accuracy_score(y_test, y_pred)
print(f"accuracy (%): {accuracy*100}")

accuracy (%): 72.06486931434021


In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[   72,    29],
       [ 4645, 12459]], dtype=int64)

In [19]:
#convert to dataframe
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,72,29
Actual 1,4645,12459


In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
classification_report = classification_report_imbalanced(y_test, y_pred)
print(classification_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.71      0.73      0.03      0.72      0.52       101
          1       1.00      0.73      0.71      0.84      0.72      0.52     17104

avg / total       0.99      0.73      0.71      0.84      0.72      0.52     17205



In [21]:
# note on "spe" in classification_report
# The specificity is the ratio tn / (tn + fp) where tn is the number of true negatives and fn the number of false negatives.

### SMOTE Oversampling

In [22]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 1)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
print(Counter(y_train))
print(Counter(y_resampled))

Counter({1: 51366, 0: 246})
Counter({1: 51366, 0: 51366})


In [23]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
# fit the logreg model to the resampled training data
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
# make prediction using y_test data
y_pred = model.predict(X_test)
accuracy = balanced_accuracy_score(y_test, y_pred)
print(f"accuracy (%): {accuracy*100}")

accuracy (%): 72.54822564810269


In [25]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[   73,    28],
       [ 4649, 12455]], dtype=int64)

In [26]:
#convert to dataframe
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,73,28
Actual 1,4649,12455


In [27]:
# Print the imbalanced classification report
classification_report = classification_report_imbalanced(y_test, y_pred)
print(classification_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.72      0.73      0.03      0.73      0.53       101
          1       1.00      0.73      0.72      0.84      0.73      0.53     17104

avg / total       0.99      0.73      0.72      0.84      0.73      0.53     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [28]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
print(Counter(y_train))
print(Counter(y_resampled))

Counter({1: 51366, 0: 246})
Counter({0: 246, 1: 246})


In [29]:
# Train the Logistic Regression model using the resampled data
# fit the logreg model to the resampled training data
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [30]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
accuracy = balanced_accuracy_score(y_test, y_pred)
print(f"accuracy (%): {accuracy*100}")

accuracy (%): 69.96606086006169


In [31]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[   75,    26],
       [ 5871, 11233]], dtype=int64)

In [32]:
#convert to dataframe
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,75,26
Actual 1,5871,11233


In [33]:
classification_report = classification_report_imbalanced(y_test, y_pred)
print(classification_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.74      0.66      0.02      0.70      0.49       101
          1       1.00      0.66      0.74      0.79      0.70      0.48     17104

avg / total       0.99      0.66      0.74      0.79      0.70      0.48     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [34]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smoteenn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)
print(Counter(y_test))
print(Counter(y_resampled))

Counter({1: 17104, 0: 101})
Counter({0: 51366, 1: 48236})


In [35]:
# Train the Logistic Regression model using the resampled data
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [36]:
# Calculated the balanced accuracy score
# make prediction using y_test data
y_pred = model.predict(X_test)
accuracy = balanced_accuracy_score(y_test, y_pred)
print(f"accuracy (%): {accuracy*100}")

accuracy (%): 71.57769243949652


In [37]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[   73,    28],
       [ 4981, 12123]], dtype=int64)

In [38]:
#convert to dataframe
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,73,28
Actual 1,4981,12123


In [39]:
# Print the imbalanced classification report
classification_report = classification_report_imbalanced(y_test, y_pred)
print(classification_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.72      0.71      0.03      0.72      0.51       101
          1       1.00      0.71      0.72      0.83      0.72      0.51     17104

avg / total       0.99      0.71      0.72      0.82      0.72      0.51     17205

